In [13]:
from akida_models.utk_face.preprocessing import load_data

# Load the dataset
x_train, y_train, x_test, y_test = load_data()

In [4]:
# For Akida inference, use uint8 raw data
x_test_akida = x_test.astype('uint8')

In [5]:
from akida_models import fetch_file
from tensorflow.keras.models import load_model

# Retrieve the model file from the BrainChip data server
model_file = fetch_file(fname="vgg_utk_face.h5",
                        origin="https://data.brainchip.com/models/AkidaV2/vgg/vgg_utk_face.h5",
                        cache_subdir='models')

# Load the native Keras pre-trained model
model_keras = load_model(model_file)
model_keras.summary()

Model: "vgg_utk_face"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 32, 32, 3)]       0         
                                                                 
 rescaling (Rescaling)       (None, 32, 32, 3)         0         
                                                                 
 conv_0 (Conv2D)             (None, 30, 30, 32)        864       
                                                                 
 conv_0/BN (BatchNormalizati  (None, 30, 30, 32)       128       
 on)                                                             
                                                                 
 conv_0/relu (ReLU)          (None, 30, 30, 32)        0         
                                                                 
 conv_1 (Conv2D)             (None, 30, 30, 32)        9216      
                                                      

In [14]:
# Compile the native Keras model (required to evaluate the MAE)
model_keras.compile(optimizer='Adam', loss='mae')

# Check Keras model performance
mae_keras = model_keras.evaluate(x_test, y_test, verbose=0)

print("Keras MAE: {0:.4f}".format(mae_keras))

Keras MAE: 6.0806


In [15]:

import quantizeml
import json
from quantizeml.models import quantize, QuantizationParams,dump_config
qparams = QuantizationParams(input_weight_bits=8, weight_bits=8, activation_bits=8, output_bits=8,
                             per_tensor_activations=True, buffer_bits=24)

# Quantize the model
quantized_model = quantize(model_keras, qparams=qparams)
quantized_model.summary()

1024/1024 [==============================] - 2s 2ms/step
Model: "vgg_utk_face"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 32, 32, 3)]       0         
                                                                 
 rescaling (QuantizedRescali  (None, 32, 32, 3)        0         
 ng)                                                             
                                                                 
 conv_0 (QuantizedConv2D)    (None, 30, 30, 32)        896       
                                                                 
 conv_0/relu (QuantizedReLU)  (None, 30, 30, 32)       2         
                                                                 
 conv_1 (QuantizedConv2D)    (None, 30, 30, 32)        9248      
                                                                 
 conv_1/maxpool (QuantizedMa  (None, 15, 15, 32)       0       

In [8]:
from akida_models import vgg_utk_face_pretrained

# Load the pre-trained quantized model
model_quantized_keras = vgg_utk_face_pretrained()
model_quantized_keras.summary()

# Compile the quantized Keras model (required to evaluate the MAE)
model_quantized_keras.compile(optimizer='Adam', loss='mae')

# Check Keras model performance
mae_quant = model_quantized_keras.evaluate(x_test, y_test, verbose=0)

print("Keras MAE: {0:.4f}".format(mae_quant))

Model: "vgg_utk_face"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 32, 32, 3)]       0         
                                                                 
 rescaling (QuantizedRescali  (None, 32, 32, 3)        0         
 ng)                                                             
                                                                 
 conv_0 (QuantizedConv2D)    (None, 30, 30, 32)        896       
                                                                 
 conv_0/relu (QuantizedReLU)  (None, 30, 30, 32)       64        
                                                                 
 conv_1 (QuantizedConv2D)    (None, 30, 30, 32)        9248      
                                                                 
 conv_1/maxpool (QuantizedMa  (None, 15, 15, 32)       0         
 xPool2D)                                             

In [16]:
from cnn2snn import convert

model_akida = convert(model_quantized_keras)
model_akida.summary()

                Model Summary                 
______________________________________________
Input shape  Output shape  Sequences  Layers
[32, 32, 3]  [1, 1, 1]     1          8     
______________________________________________

_________________________________________________________
Layer (type)               Output shape  Kernel shape  

============ SW/conv_0-dequantizer (Software) ===========

conv_0 (InputConv2D)       [30, 30, 32]  (3, 3, 3, 32) 
_________________________________________________________
conv_1 (Conv2D)            [15, 15, 32]  (3, 3, 32, 32)
_________________________________________________________
conv_2 (Conv2D)            [15, 15, 64]  (3, 3, 32, 64)
_________________________________________________________
conv_3 (Conv2D)            [8, 8, 64]    (3, 3, 64, 64)
_________________________________________________________
conv_4 (Conv2D)            [1, 1, 84]    (3, 3, 64, 84)
_________________________________________________________
dense_1 (Dense2D)       

In [17]:
import numpy as np

def preprocess_frame(frame):
    # Convert frame to RGB if necessary
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Resize frame to the model input size (32, 32, 3)
    resized_frame = cv2.resize(rgb_frame, (32, 32))
    # Normalize the frame
    #normalized_frame = resized_frame / 255.0
    #convert to uint8
    reduced_frame = resized_frame.astype('uint8')
    # Add batch dimension
    input_frame = np.expand_dims(reduced_frame, axis=0)
    return input_frame


In [18]:
import cv2

# Initialize video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame for the model
    input_frame = preprocess_frame(frame)

    # Predict age using the Akida model
    age = model_akida.predict(input_frame)

    # Display the results on the frame
    cv2.putText(frame, f'Age: {int(age)}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.imshow('Age Estimation', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [12]:
import cv2
import numpy as np
from collections import deque

# Initialize video capture
cap = cv2.VideoCapture(0)

# Initialize deque to store the last 10 age predictions
age_buffer = deque(maxlen=10)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame for the model
    input_frame = preprocess_frame(frame)

    # Predict age using the Akida model
    age = model_akida.predict(input_frame)
    age_buffer.append(age)

    # Calculate the average age
    avg_age = np.mean(age_buffer)

    # Display the results on the frame
    cv2.putText(frame, f'Age: {int(avg_age)}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.imshow('Age Estimation', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
